In [ ]:
import sys
import os
from os.path import exists

import json

from sklearn.metrics import r2_score

In [ ]:
def match_json_files(folder1, folder2):
    matched_files = []

    if not os.path.isdir(folder1) or not os.path.isdir(folder2):
        raise ValueError("Both inputs must be valid directories.")

    folder1_files = {f for f in os.listdir(folder1) if f.endswith(".json")}
    folder2_files = {f for f in os.listdir(folder2) if f.endswith(".json")}

    common_files = folder1_files.intersection(folder2_files)

    for file in common_files:
        path1 = os.path.join(folder1, file)
        path2 = os.path.join(folder2, file)
        matched_files.append((path1, path2))

    return matched_files

def load_validation_data(file_path, translations : list, rotations: list):
    with open(file_path, 'r') as f:
        data = json.load(f)

    object_in_image = False

    for i_line in range(len(data['objects'])):
        info = data['objects'][i_line]

        if "Ketchup".lower() not in info['class'].lower():
            continue                     

        # Parse translations
        location = info.get('location', [0, 0, 0])
        translations.extend(location)

        # Parse quaternion
        rot = info.get("quaternion_xyzw", [0, 0, 0, 1])
        rotations.extend(rot)
        object_in_image = True
    
    if not object_in_image:
        translations.extend([0,0,0])
        rotations.extend([0,0,0,1])     

In [ ]:
validation_folder = r"C:\github\POSEIDON\other\test_frame_images"
predictions_folder = r"C:\github\POSEIDON\output\net_batchweights_ketchup_500"

matched_files = match_json_files(validation_folder, predictions_folder)

val_translations = []
val_rotations = []

pred_translations = []
pred_rotations = []

for i, (validation_file, prediction_file) in enumerate(matched_files):
    #print(f"Processing file {i + 1}/{len(matched_files)}: {validation_file} vs {prediction_file}")

    load_validation_data(validation_file, val_translations, val_rotations)
    load_validation_data(prediction_file, pred_translations, pred_rotations)

pred_translations = [x / 100 for x in pred_translations]

print("Validation Translations:", len(val_translations))
print("Validation Rotations:", len(val_rotations))
print("Prediction Translations:", len(pred_translations))
print("Prediction Rotations:", len(pred_rotations))


print("Validation Translations:", val_translations)
print("Validation Rotations:", val_rotations)
print("Prediction Translations:", pred_translations)
print("Prediction Rotations:", pred_rotations)

In [ ]:
# Example vectors (replace these with your actual data)

# Calculate R² score
r2_translation = r2_score(val_translations, pred_translations)
r2_rotation = r2_score(val_rotations, pred_rotations)
print("R² score for translations:", r2_translation)
print("R² score for rotations:", r2_rotation)